# Each generation of iPhone has its own slogan. The slogan for iPhone 12 is "It's a leap year"

![title](img/iphone12_slogan.png)


# Do you know the slogans for all previous generations of iPhones?

# In this notebook, i will find out slogans for each generation of iPhones by looking through wikipedia page (with BeautifulSoup)


In [1]:
import pandas as pd
import requests
import re
import time
from bs4 import BeautifulSoup as bs

In [2]:
display(HTML("<style>.container { width:100% !important; }</style>"))
css = open('../style-table.css').read() + open('../style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

# iPhone main wiki page

In [3]:
url = 'https://en.wikipedia.org/wiki/IPhone'

response = requests.get(url)

response.status_code

200

In [4]:
page = response.text

soup = bs(page)

In [5]:
mytable = soup.find('table', class_ = 'wikitable')

# Get wiki page for each generation of iPhone

In [6]:
iphone_re = re.compile('iPhone')

wiki_prefix = 'https://en.wikipedia.org'

iphone_urls = [ wiki_prefix + p.find(text = iphone_re).parent['href'] 
               for p in mytable.find_all('th') 
               if (p.find(text = iphone_re))]

iphone_urls

['https://en.wikipedia.org/wiki/IPhone_(1st_generation)',
 'https://en.wikipedia.org/wiki/IPhone_3G',
 'https://en.wikipedia.org/wiki/IPhone_3GS',
 'https://en.wikipedia.org/wiki/IPhone_4',
 'https://en.wikipedia.org/wiki/IPhone_4S',
 'https://en.wikipedia.org/wiki/IPhone_5',
 'https://en.wikipedia.org/wiki/IPhone_5C',
 'https://en.wikipedia.org/wiki/IPhone_5S',
 'https://en.wikipedia.org/wiki/IPhone_6',
 'https://en.wikipedia.org/wiki/IPhone_6S',
 'https://en.wikipedia.org/wiki/IPhone_SE_(1st_generation)',
 'https://en.wikipedia.org/wiki/IPhone_7',
 'https://en.wikipedia.org/wiki/IPhone_8',
 'https://en.wikipedia.org/wiki/IPhone_X',
 'https://en.wikipedia.org/wiki/IPhone_XS',
 'https://en.wikipedia.org/wiki/IPhone_XR',
 'https://en.wikipedia.org/wiki/IPhone_11',
 'https://en.wikipedia.org/wiki/IPhone_11_Pro',
 'https://en.wikipedia.org/wiki/IPhone_SE_(2nd_generation)',
 'https://en.wikipedia.org/wiki/IPhone_12',
 'https://en.wikipedia.org/wiki/IPhone_12_Pro']

# Get slogan by visiting each url

In [7]:
def get_page_from_url(url):
    page = requests.get(url).text
    return bs(page)

In [8]:
slogans = []
iPhone_names = []
for url in iphone_urls:
    page = get_page_from_url(url)
    time.sleep(1)
    
    iPhone_names.append(page.find('caption', class_='fn').text)
    slogans.append(page.find(title = 'Slogan').next.next.text)

# Make DataFrame and adjust appearance

In [9]:
df = pd.DataFrame({'iPhone' : iPhone_names,  'Slogan' : slogans})
df.head(2)

,iPhone,Slogan
0,iPhone,"\n""This is only the beginning.""\n""Apple reinve..."
1,iPhone 3G,"""The iPhone you have been waiting for.""""The fi..."


In [10]:
df.Slogan = df.Slogan.apply(lambda x: re.sub('"', '', x)).apply(lambda x: re.sub(r'\[\d+\]', '', x)).apply(lambda x: re.sub(r'\n', '', x))
df.head(2)

,iPhone,Slogan
0,iPhone,This is only the beginning.Apple reinvents the...
1,iPhone 3G,The iPhone you have been waiting for.The first...


In [11]:
pd.set_option('display.max_colwidth', 200)
dfStyler = df.style.set_properties(**{'text-align': 'left'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

,iPhone,Slogan
0,iPhone,This is only the beginning.Apple reinvents the phone.
1,iPhone 3G,"The iPhone you have been waiting for.The first phone to beat the iPhone.Twice as fast, for half the price.The most advanced mobile OS. Now even more advanced.New features, new price."
2,iPhone 3GS,"The fastest, smartest phone yet. (2009–2010)More to love. Less to pay. (2010–2012)"
3,iPhone 4,This changes everything. Again.
4,iPhone 4S,The most amazing iPhone yet.
5,iPhone 5,Loving it is easy. That's why so many people do. (since March 2013)The biggest thing to happen to iPhone since iPhone. (before March 2013)
6,iPhone 5c,For the colorful
7,iPhone 5S,Since April 2014: You're more powerful than you think.Until April 2014: Forward thinking
8,iPhone 6iPhone 6 Plus,Bigger than biggerThe two and only
9,iPhone 6SiPhone 6S Plus,The only thing that’s changed is everything.One powerful phone
